# Demo DERIVA Ingest Action Provider Flow
This notebook demonstrates using the DERIVA Ingest Action Provider via a premade Flow.

To run this notebook, you must have the `globus-automate-client` installed. The client can be installed with `pip`:

`pip install globus-automate-client`

In [1]:
import globus_automate_client
# Prerequisite IDs and data
native_app_id = "417301b1-5101-456a-8a27-423e71a2ae26"  # Premade native app ID
flow_id = "f172de09-b75b-4b83-9b97-90877b42c774"  # ID for flow to use, can be found through .list_flows()
backup_url = "https://s3-us-west-2.amazonaws.com/demo.derivacloud.org/cfde/cfde-backup.zip"  # Data for the Action

The `FlowsClient` is instantiated with the client ID of a native app registered with Globus (see the [Globus Developers](https://developers.globus.org/) website).

In [2]:
flows_client = globus_automate_client.create_flows_client(native_app_id)

The definition of a Flow can be found by querying on the Flow's ID. This definition includes the Flow's scope, which is needed for running the Flow.

In [3]:
flow_def = flows_client.get_flow(flow_id)
flow_def

GlobusHTTPResponse({'administered_by': [], 'api_version': '1.0', 'definition': {'Comment': 'Run the Demo Deriva Ingest Action', 'StartAt': 'RunDeriva', 'States': {'RunDeriva': {'ActionScope': 'https://auth.globus.org/scopes/21017803-059f-4a9b-b64c-051ab7c1d05d/demo', 'ActionUrl': 'https://demo-api.fair-research.org/', 'End': True, 'InputPath': '$.DerivaInput', 'ResultPath': '$.DerivaResult', 'Type': 'Action', 'WaitTime': 86400}}}, 'description': '', 'globus_auth_scope': 'https://auth.globus.org/scopes/f172de09-b75b-4b83-9b97-90877b42c774/flow_f172de09_b75b_4b83_9b97_90877b42c774', 'globus_auth_username': 'f172de09-b75b-4b83-9b97-90877b42c774@clients.auth.globus.org', 'id': 'f172de09-b75b-4b83-9b97-90877b42c774', 'keywords': [], 'log_supported': True, 'principal_urn': 'urn:globus:auth:identity:f172de09-b75b-4b83-9b97-90877b42c774', 'runnable_by': [], 'subtitle': '', 'synchronous': False, 'title': 'Deriva Demo Flow', 'types': ['Action'], 'visible_to': []})

In [4]:
flow_scope = flow_def["globus_auth_scope"]

Input keyed on the Action name (see the `flow_def` above) is passed to the Action Provider directly.

In [5]:
flow_input = {
    "DerivaInput": {
        "url": backup_url
    }
}

Running the flow combines all the previous elements into one call, and returns the starting state of the Flow, including the ID of this instance of the Flow.

In [6]:
flow_res = flows_client.run_flow(flow_id, flow_scope, flow_input)
flow_res

Login Here:

https://auth.globus.org/v2/oauth2/authorize?client_id=417301b1-5101-456a-8a27-423e71a2ae26&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ff172de09-b75b-4b83-9b97-90877b42c774%2Fflow_f172de09_b75b_4b83_9b97_90877b42c774&state=_default&response_type=code&code_challenge=0JFcWrGIzzCWH5Bu4TGAf70l7EMmngOvFfYXqrwc8_0&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Client


Note that this link can only be used once! If login or a later step in the flow fails, you must restart it.
Enter resulting code:gr05Layp87kqr331PvQ8lVtocq5ktL


GlobusHTTPResponse({'completion_time': 'None', 'details': {'code': 'ActionStarted', 'description': 'State RunDeriva of type Action started', 'details': {'input': {'DerivaInput': {'url': 'https://s3-us-west-2.amazonaws.com/demo.derivacloud.org/cfde/cfde-backup.zip'}}, 'state_name': 'RunDeriva', 'state_type': 'Action'}, 'time': '2019-08-15T19:39:20.111000+00:00'}, 'id': 'da83f283-34e9-4905-993a-31dd90e3b34b', 'start_time': '2019-08-15T19:39:20.085000+00:00', 'status': 'ACTIVE'})

In [7]:
instance_id = flow_res["id"]

After the Flow is started, the status can be queried with the `instance_id`.

Note: For some reason, the Flow status can start as "failed" when the Action is still in progress. This does not mean the Action or Flow has failed, unless the `status` actually changes to `FAILED`.

In [12]:
flows_client.flow_action_status(flow_id, flow_scope, instance_id)

GlobusHTTPResponse({'completion_time': '2019-08-15T19:41:41.559000+00:00', 'details': {'output': {'DerivaInput': {'url': 'https://s3-us-west-2.amazonaws.com/demo.derivacloud.org/cfde/cfde-backup.zip'}, 'DerivaResult': {'action_id': '5d55b4ec3b34c022ef7efaad', 'creator_id': 'urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1', 'details': {'deriva_id': 104, 'deriva_samples_link': 'https://demo.derivacloud.org/chaise/recordset/#104/demo:Samples', 'message': 'DERIVA restore successful'}, 'manage_by': ['urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1', 'urn:globus:auth:identity:96801dc2-95d7-44f0-9383-d3e747be8ab6'], 'monitor_by': ['urn:globus:auth:identity:117e8833-68f5-4cb2-afb3-05b25db69be1', 'urn:globus:auth:identity:96801dc2-95d7-44f0-9383-d3e747be8ab6'], 'release_after': 'P30D', 'request_id': 'flows_req_4a3ad5c3-c07b-4edc-8894-38a178d7dbc1', 'status': 'SUCCEEDED'}}}, 'id': 'da83f283-34e9-4905-993a-31dd90e3b34b', 'start_time': '2019-08-15T19:39:20.085000+00:00'